In [1]:
pip install pandas numpy scikit-learn surprise nltk textblob flask streamlit

In [2]:
!pip uninstall -y numpy
!pip install numpy==1.23.5
!pip install scikit-surprise


Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which 

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from textblob import TextBlob
import random

recipes_url = "https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv"
ratings_url = "https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv"

recipes = pd.read_csv(recipes_url)
ratings = pd.read_csv(ratings_url)

# Rename columns
recipes.rename(columns={'book_id': 'id', 'title': 'name', 'authors': 'ingredients'}, inplace=True)
ratings.rename(columns={'book_id': 'recipe_id', 'user_id': 'user_id', 'rating': 'rating'}, inplace=True)

# Preprocessing
recipes.dropna(subset=["ingredients"], inplace=True)
ratings.dropna(inplace=True)

# TF-IDF Vectorization for Content-Based Filtering
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(recipes['ingredients'].astype(str))
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend similar recipes
def recommend_recipes(recipe_id, top_n=5):
    if recipe_id not in recipes['id'].values:
        return "Recipe ID not found."

    idx = recipes[recipes['id'] == recipe_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    recipe_indices = [i[0] for i in sim_scores]
    return recipes.iloc[recipe_indices][['id', 'name']]

# Collaborative Filtering (Matrix Factorization)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'recipe_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
model = SVD()
model.fit(trainset)

# Function to predict ratings
def predict_rating(user_id, recipe_id):
    return model.predict(user_id, recipe_id).est

# Generating synthetic location & weather data
locations = ["New York", "San Francisco", "Chicago", "Los Angeles", "Houston"]
weather_conditions = ["Sunny", "Rainy", "Cold", "Hot"]

def get_user_context(user_id):
    return {
        "location": random.choice(locations),
        "weather": random.choice(weather_conditions)
    }

# Sentiment Analysis
def analyze_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

# Context-Aware Recommendations with Sentiment Analysis
def context_aware_recommendations(user_id, user_feedback="", top_n=5):
    user_context = get_user_context(user_id)
    sentiment = analyze_sentiment(user_feedback)
    print(f"User Context: {user_context}, Sentiment: {sentiment}")

    user_rated_recipes = ratings[ratings['user_id'] == user_id]['recipe_id'].tolist()
    recommendations = []
    for recipe_id in recipes['id'].sample(20):
        if recipe_id not in user_rated_recipes:
            predicted_rating = predict_rating(user_id, recipe_id)
            recommendations.append((recipe_id, predicted_rating))

    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[:top_n]
    return recipes[recipes['id'].isin([r[0] for r in recommendations])][['id', 'name']]

# Example Usage
print(recommend_recipes(10))  # Content-Based
print(context_aware_recommendations(5, "I love reading books with deep insights!"))  # Context-Aware with Sentiment


        id                                               name
4652  4653                                The Complete Novels
6301  6302                                         Lady Susan
450    451                                   Northanger Abbey
170    171                                               Emma
1038  1039  Pride and Prejudice and Zombies (Pride and Pre...
User Context: {'location': 'New York', 'weather': 'Cold'}, Sentiment: Positive
        id                                               name
130    131                                The Grapes of Wrath
3662  3663                Squire (Protector of the Small, #3)
3735  3736  Harry Potter Page to Screen: The Complete Film...
6904  6905  Notorious RBG: The Life and Times of Ruth Bade...
7265  7266    The Wonderful Story of Henry Sugar and Six More
